In [40]:
# -----------------------------------------------------------------------------
# 1) Imports & data inlezen
# -----------------------------------------------------------------------------
import pandas as pd
import numpy as np
import networkx as nx

# PyWake-imports (zorg dat je 'py_wake' en 'py_wake.wind_turbines' hebt geïnstalleerd)
from py_wake import BastankhahGaussian
from py_wake.wind_turbines import WindTurbine
from py_wake.site import UniformWeibullSite
from py_wake.wind_turbines.power_ct_functions import PowerCtTabular


In [30]:

# Stel: je hebt een dataframe df met kolommen: ['Wdir', 'Ndir']
# Hier gebruiken we de kolommen zoals ze in jouw SCMDf dataset zitten.

def normalize_angle(angle):
    """
    Brengt een willekeurige hoek in graden terug naar [0, 360).
    """
    return angle % 360

def compute_absolute_wind_direction(df):
    """
    Bereken absolute windrichting in graden t.o.v. true north (0-360°).
    """
    # Vang eventuele NaN waardes af:
    df = df.copy()
    df['Wdir'] = df['Wdir'].fillna(0)
    df['Ndir'] = df['Ndir'].fillna(0)
    
    # Stap 1: optellen van Ndir en Wdir
    df['AbsWdir'] = df['Ndir'] + df['Wdir']
    
    # Stap 2: normaliseren naar 0–360 graden
    df['AbsWdir'] = df['AbsWdir'].apply(normalize_angle)
    
    return df

# Voorbeeld: pas op je eigen dataframe toe
df = compute_absolute_wind_direction(df)

# Controle
print(df[['Wdir', 'Ndir', 'AbsWdir']].head())


   Wdir   Ndir  AbsWdir
0  0.00   0.00     0.00
1 -3.99  25.92    21.93
2 -2.18  20.91    18.73
3 -0.73  20.91    20.18
4  0.89  20.91    21.80


In [15]:
# 1.1) Lees turbine‐locaties
#     (turbine_location.CSV bevat minstens: TurbID, x_coord, y_coord)
df_loc = pd.read_csv('GML/data/turbine_location.csv') 
# Veronderstel kolommen: ['TurbID', 'x', 'y']
# (pas kolomnamen aan indien anders)
print("Voorbeeld turbine-locaties:")
print(df_loc.head())

# 1.2) Lees je wind‐ en vermogensdata 
#     (wind_power_sdwpf.csv heeft o.a. kolommen: TurbID, Datetime, Wspd, Wdir, Patv, etc.)
# df = pd.read_csv('GML/data/wind_power_sdwpf.csv', parse_dates=['Day'])
df = pd.read_csv('GML/data/wind_power_sdwpf.csv')
print("\nVoorbeeld wind‐data:")
print(df.head())

# 1.3) Merge locaties met wind‐meting
#     Omdat je voor PyWake alleen x/y en per tijdstip Wdir/Wspd nodig hebt:
#     maak een DataFrame waarin per tijdstip (Datetime) en per TurbID 
#     de Wspd, Wdir gecombineerd staan met de (x,y)-coördinaten.
df_merged = df.merge(df_loc, on='TurbID', how='left')
# Let op: dit wordt (TurbCount × TimeSteps) regels groot. 
#        Per tijdstap heb je echter Wspd, Wdir voor élke turbine.  

Voorbeeld turbine-locaties:
   TurbID          x           y
0       1  3349.8515  5939.23193
1       2  3351.0017  6416.64673
2       3  3314.7797  6892.18395
3       4  3352.0940  7366.14203
4       5  3355.3420  7841.20175

Voorbeeld wind‐data:
   TurbID  Day Tmstamp  Wspd  Wdir   Etmp   Itmp   Ndir  Pab1  Pab2  Pab3  \
0       1    1   00:00   NaN   NaN    NaN    NaN    NaN   NaN   NaN   NaN   
1       1    1   00:10  6.17 -3.99  30.73  41.80  25.92   1.0   1.0   1.0   
2       1    1   00:20  6.27 -2.18  30.60  41.63  20.91   1.0   1.0   1.0   
3       1    1   00:30  6.42 -0.73  30.52  41.52  20.91   1.0   1.0   1.0   
4       1    1   00:40  6.25  0.89  30.49  41.38  20.91   1.0   1.0   1.0   

   Prtv    Patv  
0   NaN     NaN  
1 -0.25  494.66  
2 -0.24  509.76  
3 -0.26  542.53  
4 -0.23  509.36  


In [32]:
# 1.4) Kies één tijdstap om mee te beginnen (bijv. '2020-01-01 00:10:00')
# Filter for day 1 and timestamp '00:10:00'
df_t0 = df[(df['Day'] == 1) & (df['Tmstamp'].str.startswith('00:10'))].sort_values('TurbID')

print(f"Aantal turbines op dag 1, tijd 00:10: {len(df_t0)}")
print(df_t0[['TurbID', 'Wspd', 'AbsWdir']].head())

# Merge met locatie-data om x/y toe te voegen
df_t0 = df_t0.merge(df_loc, on='TurbID', how='left')

# Maak numpy‐arrays van Wspd, Wdir, x, y:
ws_vec = df_t0['Wspd'].values   # shape = (n_turbines,)
wd_vec = df_t0['AbsWdir'].values   # shape = (n_turbines,)
x_vec  = df_t0['x'].values      # shape = (n_turbines,)
y_vec  = df_t0['y'].values      # shape = (n_turbines,)

Aantal turbines op dag 1, tijd 00:10: 134
        TurbID  Wspd  AbsWdir
1            1  6.17    21.93
35281        2  6.85    18.52
70561        3  6.36   270.20
105841       4  5.65    27.98
141121       5  6.01   345.86


In [48]:

# definieer windsnelheden (b.v. 0 tot 25 m/s met stappen van 1 m/s)
ws_array = np.arange(0, 26, 1)

# een simpele dummy power-curve:
power_array = np.clip((ws_array - 3) * (1500 / (12 - 3)), 0, 1500)

# dummy constant ct
ct_array = np.full_like(ws_array, 0.8)

# Nu maak je het correcte powerCtFunction object
powerCtFunction = PowerCtTabular(ws=ws_array, power=power_array, ct=ct_array, power_unit='kW')


# Nu pas maak je je turbine-object (LET OP: alleen powerCtFunction gebruiken)
turbine = WindTurbine(
    name="MyTurbine",
    diameter=80,
    hub_height=80,
    powerCtFunction=powerCtFunction
)


In [59]:
# 2.2) Zet de site op: UniformWeibullSite dient als placeholder omdat we later gemeten Wspd/Wdir injecteren

# Aantal sectoren (standaard: 360 graden)
n_sectors = 360

# Uniforme verdeling over alle sectoren
p_wd = np.ones(n_sectors) / n_sectors

# Dummy Weibull parameters per sector (dezelfde waarden voor alle sectoren)
a = np.full(n_sectors, 2.0)   # scale parameter
k = np.full(n_sectors, 2.0)   # shape parameter
ti = np.full(n_sectors, 0.10) # turbulentie-intensiteit

# Bouw de UniformWeibullSite
site = UniformWeibullSite(
    p_wd=p_wd,
    a=a,
    k=k,
    ti=ti,
    distance= None
)




In [49]:
wfm = BastankhahGaussian(site, turbine)

c:\Users\daank\GML\venv\Lib\site-packages\py_wake\deficit_models\gaussian.py:124: UserWarning: The BastankhahGaussian model is not representative of the setup used in the literature. For this, use py_wake.literature.gaussian_models.Bastankhah_PorteAgel_2014 instead
  DeprecatedModel.__init__(self, 'py_wake.literature.gaussian_models.Bastankhah_PorteAgel_2014')


In [54]:
type(x_vec)

print(f"x_vec shape: {x_vec.shape}, dtype: {x_vec.dtype}")
print(f"y_vec shape: {y_vec.shape}, dtype: {y_vec.dtype}")


x_vec shape: (134,), dtype: float64
y_vec shape: (134,), dtype: float64


In [52]:
# -----------------------------------------------------------------------------
# 3) Wake‐simulatie uitvoeren voor tijdstip t0
# -----------------------------------------------------------------------------
# PyWake verwacht dat Wdir en Wspd als “lists” of 1D‐arrays binnenkomen:
#   - wd = [waarde in graden]
#   - ws = [waarde in m/s]
#
# En x,y-coördinaten als arrays van lengte n_turbines.
wd_t0 = df_t0['AbsWdir'].mean()
ws_t0 = df_t0['Wspd'].mean()

x_vec_3d = x_vec.reshape((1, 1, -1))  # shape = (1, 1, n_turbines)
y_vec_3d = y_vec.reshape((1, 1, -1))  # shape = (1, 1, n_turbines)

sim_res = wfm(x_vec_3d, y_vec_3d, wd=[wd_t0], ws=[ws_t0])

# NB: we nemen hier aan dat alle turbines op t0 dezelfde Wdir en Wspd hebben. 
#     Als je _per turbine_ verschillende Wdir/Wspd wilt, kun je per turbine
#     _meerdere_ posities simuleren, of er is een uitgebreidere Site‐setup nodig.
#
# De methode hierboven (“wd=[wd0], ws=[ws0]”) is puur bedoeld om te laten zien
# hoe je één uniforme windvoorwaarde doorgeeft. 
#
# Als je écht per turbine verschillende Wdir/​Wspd hebt, zou je voor elke turbine
# apart een simulatie moeten draaien (met dezelfde set x_vec/y_vec), 
# en dan de resulterende efficiency‐waarden extraheren. Dat kan langzamer lopen,
# maar is in principe mogelijk.

ValueError: Argument, x(shape=(1, 1, 134)), has unsupported shape. Valid shapes are (), (1), (1,1), (1,1,1), (1,), (1, 1) (interpreted in this order)